# Neural Network Learning

Initial imports

In [23]:
import numpy as np
import nnfs
import matplotlib.pyplot as plt
from nnfs.datasets import spiral_data
nnfs.init()

Neural network layer class

In [29]:
class Layer_Dense:
    
    # Neuron initialization
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    # Neuron forward pass for calculations
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass for derivative backpropagation
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

Rectified Linear Unit (ReLU) activation function

In [35]:
class Activation_ReLU:
    
    # Forward pass for output calculations
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    # Backward pass for derivative backpropagation
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

Softmax Activation function

In [20]:
class Activation_Softmax:
    
    # Forward pass for output
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    # Backward pass for derivative backpropagation
    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)

        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

Loss parent class

In [18]:
class Loss:
    
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

Loss calculation function

In [17]:
class Loss_CategorialCrossentropy(Loss):

    # Forward pass for loss calculation output
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis = 1)
        
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    # Backward pass for derivative backpropagation
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

Combined softmax activation and cross entropy loss for faster backward step

In [26]:
class Activation_Softmax_Loss_CategorialCrossentropy():


    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategorialCrossentropy()

    
    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)
    

    def backward(self, dvalues, y_true):
        samples = len(dvalues)

        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs / samples

Code runner

In [49]:
X, y = spiral_data(samples=100, classes=3)

dense1 = Layer_Dense(2, 3)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(3, 3)
loss_activation = Activation_Softmax_Loss_CategorialCrossentropy()

dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)

loss = loss_activation.forward(dense2.output, y)

print(loss_activation.output[:5])

print('loss: ', loss)

predictions = np.argmax(loss_activation.output, axis = 1)
if len(y.shape) == 2:
    y = np.argmax(y, axis = 1)
accuracy = np.mean(predictions == y)

print('acc: ', accuracy)

loss_activation.backward(loss_activation.output, y)
dense2.backward(loss_activation.dinputs)
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

# Print gradients
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)

[[0.33333334 0.33333334 0.33333334]
 [0.33333415 0.33333343 0.33333242]
 [0.3333348  0.33333346 0.3333317 ]
 [0.33333498 0.33333337 0.33333164]
 [0.333336   0.3333335  0.33333045]]
loss:  1.0986096
acc:  0.37666666666666665
[[ 3.0099418e-05  1.2299024e-04 -1.9024039e-04]
 [ 6.1281004e-05  3.4426223e-04 -8.2684142e-05]]
[[ 3.3247350e-05 -1.7246224e-04 -3.1159539e-04]]
[[ 8.3425570e-05 -8.3962172e-05  5.3659852e-07]
 [-2.8484732e-05  2.6014573e-05  2.4701590e-06]
 [ 3.9678551e-05 -2.5594767e-04  2.1626914e-04]]
[[ 1.0024407e-05 -1.0471558e-06 -9.0627000e-06]]
